#### Model Structure
- XGB Classifier
- Fine tuning

In [ ]:
import json, gzip
import numpy as np
import xgboost as xgb
import pandas as pd
from pathlib import Path
from sklearn.metrics import average_precision_score, roc_auc_score
from sklearn.model_selection import GroupShuffleSplit
import optuna
from sklearn.model_selection import StratifiedGroupKFold
import joblib

In [2]:
def parse_json_line(obj):
    (tid, tdata), = obj.items()
    (pos_key, contexts), = tdata.items()
    pos = int(pos_key) if isinstance(pos_key, str) else pos_key
    (ctx7, reads), = contexts.items()
    arr = np.asarray(reads, dtype=float)
    return tid, pos, ctx7, arr

BASE_IDX = {"A":0, "C":1, "G":2, "T":3}
def onehot28(seq7: str) -> np.ndarray:
    out = np.zeros((7,4), dtype=np.int8)
    s = (seq7 or "").upper()
    for i in range(min(7, len(s))):
        j = BASE_IDX.get(s[i], -1)
        if j >= 0:
            out[i, j] = 1
    return out.ravel()

def aggregate_9(arr: np.ndarray) -> np.ndarray:
    if arr.size == 0:
        return np.zeros(45, dtype=np.float32)
    mean = arr.mean(axis=0)
    std  = arr.std(axis=0, ddof=0)
    mn   = arr.min(axis=0)
    mx   = arr.max(axis=0)
    med  = np.median(arr, axis=0)
    return np.concatenate([mean, std, mn, mx, med]).astype(np.float32, copy=False)

NUM_COLS = [
    "dwell_m1","sd_m1","mean_m1",
    "dwell_0","sd_0","mean_0",
    "dwell_p1","sd_p1","mean_p1",
]
FEATURE_NAMES = (
    [f"mean_{c}"   for c in NUM_COLS] +
    [f"std_{c}"    for c in NUM_COLS] +
    [f"min_{c}"    for c in NUM_COLS] +
    [f"max_{c}"    for c in NUM_COLS] +
    [f"median_{c}" for c in NUM_COLS] +
    [f"ctx_{i}"    for i in range(28)]
)


def build_dataset_from_json_objects(json_objects, label_dict, transcript_to_gene):
    """
    json_objects: iterable of parsed per-line dicts
    label_dict: {(transcript_id, position): 0/1}
    Returns X (N,73), y (N,), plus ids for later mapping.
    """
    X_rows, y_rows, ids = [], [], []
    for obj in json_objects:
        tid, pos, ctx7, arr = parse_json_line(obj)
        feats45 = aggregate_9(arr)
        ctx28   = onehot28(ctx7)
        X_rows.append(np.concatenate([feats45, ctx28]))
        y_rows.append(label_dict.get((tid, int(pos)), None))
        gene = transcript_to_gene.get((tid, None))
        ids.append((gene, tid, int(pos)))
    X = np.asarray(X_rows, dtype=np.float32)
    y = np.asarray(y_rows)
    return X, y, ids

def make_data_splits(X, y, ids, test_size=0.3, val_size=0.5, random_state=4262):
    mask = ~pd.isna(y)
    X_tr = X[mask]
    y_tr = y[mask].astype(int)

    groups = np.array([
        str(gene) if gene is not None else tid
        for gene, tid, pos in ids
    ])[mask]

    gss = GroupShuffleSplit(test_size=test_size, random_state=random_state)
    train_idx, temp_idx = next(gss.split(X_tr, y_tr, groups=groups))

    gss2 = GroupShuffleSplit(test_size=val_size, random_state=random_state)
    val_idx, test_idx = next(gss2.split(X_tr[temp_idx], y_tr[temp_idx], groups=groups[temp_idx]))

    train_genes = set(groups[train_idx])
    val_genes   = set(groups[temp_idx][val_idx])
    test_genes  = set(groups[temp_idx][test_idx])
    assert len(train_genes & val_genes) == 0, "Gene overlap between train and val!"
    assert len(train_genes & test_genes) == 0, "Gene overlap between train and test!"
    assert len(val_genes & test_genes) == 0, "Gene overlap between val and test!"
    print(f"Split complete: {len(train_genes)} train genes, {len(val_genes)} val genes, {len(test_genes)} test genes")

    return {
        "X_train": X[train_idx], "y_train": y[train_idx],
        "X_val": X[temp_idx][val_idx], "y_val": y[temp_idx][val_idx],
        "X_test": X[temp_idx][test_idx], "y_test": y[temp_idx][test_idx],
        "ids_train": [ids[i] for i in train_idx],
        "ids_val": [ids[i] for i in temp_idx[val_idx]],
        "ids_test": [ids[i] for i in temp_idx[test_idx]]
    }
    

def iter_json_lines(path: str):
    """Stream NDJSON from .json or .json.gz"""
    p = Path(path)
    if p.suffix == ".gz":
        with gzip.open(p, "rt", encoding="utf-8", errors="replace") as f:
            for line in f:
                s = line.strip()
                if s:
                    yield json.loads(s)
    else:
        with open(p, "r", encoding="utf-8") as f:
            for line in f:
                s = line.strip()
                if s:
                    yield json.loads(s)

In [3]:
df_labels = pd.read_csv("data_task1/data.info.labelled.csv")
label_dict = { (r.transcript_id, int(r.transcript_position)): int(r.label) for r in df_labels.itertuples(index=False)}
transcript_to_gene = dict(zip(df_labels['transcript_id'], df_labels['gene_id']))

train_json_iter = iter_json_lines("data_task1/dataset0.json")
X, y, ids = build_dataset_from_json_objects(train_json_iter, label_dict, transcript_to_gene)
data = make_data_splits(X, y, ids)

X_train = data['X_train']
X_val = data['X_val']
X_test = data['X_test']

y_train = data['y_train']
y_val = data['y_val']
y_test = data['y_test']

ids_train = data['ids_train']
ids_val = data['ids_val']
ids_test = data['ids_test']

Split complete: 3733 train genes, 800 val genes, 800 test genes


### Fine Tuning

In [4]:
def train_and_validate_optuna(X_train, y_train, X_val, y_val, n_trials=40):
    def objective(trial):
        params = {
            "max_depth": trial.suggest_int("max_depth", 4, 7),
            "learning_rate": trial.suggest_float("learning_rate", 0.03, 0.1, log=True),
            "n_estimators": trial.suggest_int("n_estimators", 400, 800),
            "subsample": trial.suggest_float("subsample", 0.7, 0.95),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.7, 0.95),
            "scale_pos_weight": trial.suggest_int("scale_pos_weight", 10, 30),
            "min_child_weight": trial.suggest_int("min_child_weight", 2, 6),
            "gamma": trial.suggest_float("gamma", 0.05, 0.3),
            "reg_alpha": trial.suggest_float("reg_alpha", 0, 1.5),
            "reg_lambda": trial.suggest_float("reg_lambda", 2, 8),

            "tree_method": "hist",
            "eval_metric": "aucpr",
            "random_state": 4262,
            "n_jobs": -1,
        }

        clf = xgb.XGBClassifier(**params)
        clf.fit(X_train, y_train)

        y_pred = clf.predict_proba(X_val)[:, 1]
        auprc = average_precision_score(y_val, y_pred)
        auroc = roc_auc_score(y_val, y_pred)
        score = 0.7 * auprc + 0.3 * auroc

        trial.set_user_attr("auprc", auprc)
        trial.set_user_attr("auroc", auroc)
        return score

    study = optuna.create_study(direction="maximize", study_name="xgb_optuna_cpu")
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

    best_params = study.best_params
    print(f"Best params: {best_params}")
    print(f"AUPRC: {study.best_trial.user_attrs['auprc']:.4f}, AUROC: {study.best_trial.user_attrs['auroc']:.4f}")
    return best_params


In [5]:
def predict_with_model(model, X_test, y_test=None):
    y_pred = model.predict_proba(X_test)[:, 1]
    if y_test is not None:
        auprc = average_precision_score(y_test, y_pred)
        auroc = roc_auc_score(y_test, y_pred)
        print(f"Test AUPRC: {auprc:.4f}, AUROC: {auroc:.4f}")
        return y_pred, auprc, auroc
    return y_pred

In [6]:
def train_final_groupkfold_with_oof(X, y, ids, params, n_splits=5):
    groups = np.array([gene if gene is not None else tid for gene, tid, pos in ids])
    gkf = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state=4262)

    models = []
    oof_pred = np.zeros(len(y), dtype=float)
    fold_metrics = []

    for fold, (tr_idx, va_idx) in enumerate(gkf.split(X, y, groups=groups), 1):
        X_tr, y_tr = X[tr_idx], y[tr_idx]
        X_va, y_va = X[va_idx], y[va_idx]

        clf = xgb.XGBClassifier(
            **params,
            tree_method="hist", 
            eval_metric="aucpr", random_state=42 + fold, n_jobs=-1
        )
        clf.fit(
            X_tr, y_tr,
            eval_set=[(X_va, y_va)],
            verbose=False
        )
        models.append(clf)

        oof_scores = clf.predict_proba(X_va)[:, 1]
        oof_pred[va_idx] = oof_scores

        auprc = average_precision_score(y_va, oof_scores)
        auroc = roc_auc_score(y_va, oof_scores)
        fold_metrics.append((auprc, auroc))
        print(f"Fold {fold}: AUPRC={auprc:.4f}, AUROC={auroc:.4f}")

    oof_auprc = average_precision_score(y, oof_pred)
    oof_auroc = roc_auc_score(y, oof_pred)
    print(f"OOF AUPRC={oof_auprc:.4f}, OOF AUROC={oof_auroc:.4f}")

    return models, oof_pred, fold_metrics


In [7]:
def predict_ensemble(models, X):
    preds = np.zeros(X.shape[0])
    for clf in models:
        preds += clf.predict_proba(X)[:, 1]
    preds /= len(models)
    return preds

In [8]:
best_params = train_and_validate_optuna(X_train, y_train, X_val, y_val, n_trials=40)

[I 2025-11-02 19:53:58,250] A new study created in memory with name: xgb_optuna_cpu


  0%|          | 0/40 [00:00<?, ?it/s]

[I 2025-11-02 19:54:02,993] Trial 0 finished with value: 0.6024995318345401 and parameters: {'max_depth': 7, 'learning_rate': 0.08229449313595374, 'n_estimators': 622, 'subsample': 0.8390990489676048, 'colsample_bytree': 0.7396569500445129, 'scale_pos_weight': 14, 'min_child_weight': 3, 'gamma': 0.284975859761654, 'reg_alpha': 1.4584978200320429, 'reg_lambda': 4.555576139323986}. Best is trial 0 with value: 0.6024995318345401.
[I 2025-11-02 19:54:06,859] Trial 1 finished with value: 0.6027111474053263 and parameters: {'max_depth': 5, 'learning_rate': 0.06628729842673187, 'n_estimators': 682, 'subsample': 0.7115898772553497, 'colsample_bytree': 0.8698984855539328, 'scale_pos_weight': 11, 'min_child_weight': 5, 'gamma': 0.2822536198604645, 'reg_alpha': 1.2311260763370842, 'reg_lambda': 2.7626068714338654}. Best is trial 1 with value: 0.6027111474053263.
[I 2025-11-02 19:54:09,811] Trial 2 finished with value: 0.5911229420021848 and parameters: {'max_depth': 4, 'learning_rate': 0.05367642

In [22]:
X_final = np.concatenate([X_train]) ##previously we include validation data, but now we remove it to avoid overfitting
y_final = np.concatenate([y_train])
ids_final = ids_train
models, oof_pred, _ = train_final_groupkfold_with_oof(
    X_final, y_final, ids_final, best_params, n_splits=5
)

Fold 1: AUPRC=0.4808, AUROC=0.9213
Fold 2: AUPRC=0.4753, AUROC=0.9210
Fold 3: AUPRC=0.4968, AUROC=0.9317
Fold 4: AUPRC=0.4845, AUROC=0.9216
Fold 5: AUPRC=0.4916, AUROC=0.9184
OOF AUPRC=0.4833, OOF AUROC=0.9228


In [23]:
y_pred = predict_ensemble(models, X_test)
auprc = average_precision_score(y_test, y_pred)
auroc = roc_auc_score(y_test, y_pred)
print(f"Final Test AUPRC={auprc:.4f}, AUROC={auroc:.4f}")


Final Test AUPRC=0.5273, AUROC=0.9338


In [24]:
with open("models/best_params.json", "w") as f:
    json.dump(best_params, f, indent=4)

In [25]:
Path("models/final").mkdir(parents=True, exist_ok=True)
for i, model in enumerate(models):
    joblib.dump(model, f"models/final/xgb_fold{i+1}.pkl")

In [26]:
## sanity check before proceeding
def predict_ensemble_dataset(models, data_path, label_dict=None, transcript_to_gene=None, output_csv="predictions/pred_dataset0.csv"):
    preds = np.array([m.predict_proba(X)[:,1] for m in models])
    y_pred = preds.mean(axis=0)  # mean ensemble
    print(y_pred.shape)

    pred_df = pd.DataFrame({
        "gene_id": [gene for gene, tid, pos in ids],
        "transcript_id": [tid for gene, tid, pos in ids],
        "transcript_position": [pos for gene, tid, pos in ids],
        "score": y_pred
    })
    Path(output_csv).parent.mkdir(parents=True, exist_ok=True)
    pred_df.to_csv(output_csv, index=False)
    print(f"Saved predictions to {output_csv}")
    
    if label_dict:
        mask = ~pd.isna(y)
        if mask.sum() > 0:
            y_true = y[mask].astype(int)
            y_pred_masked = y_pred[mask]
            auprc = average_precision_score(y_true, y_pred_masked)
            auroc = roc_auc_score(y_true, y_pred_masked)
            print(f"len before masking: {len(y)}; len after masking: {len(y_true)}")
            print(f"[Sanity check on labeled data] AUPRC={auprc:.4f}, AUROC={auroc:.4f}")

    return pred_df


In [27]:
data_path = "data_task1/dataset_0.json"
pred_df = predict_ensemble_dataset(models, data_path, label_dict, transcript_to_gene)

(121838,)
Saved predictions to predictions/pred_dataset0.csv
len before masking: 121838; len after masking: 121838
[Sanity check on labeled data] AUPRC=0.8648, AUROC=0.9802


In [28]:
X.shape

(121838, 73)